In [1]:
library(diversedata)
data("globalrights")
head(globalrights)

,year,country,country-code,gdp-per-capita,education-spending-gdp,same-sex-marriage,lgbtq-censorship,employment-discrimination,gender-affirming-care,legal-gender
,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2001,Algeria,DZA,11742.60,NA,Banned,Imprisonment as punishment,NA,NA,NA
2,2001,Andorra,AND,59109.02,NA,NA,NA,NA,Restricted,NA
3,2001,Argentina,ARG,21066.46,4.83,NA,NA,NA,NA,NA
4,2001,Armenia,ARM,5044.12,2.47,NA,NA,NA,"Legal, but restricted for minors","Legal, surgery required"
5,2001,Australia,AUS,44667.93,5.22,NA,Varies by region,NA,Legal,Varies by region
6,2001,Austria,AUT,54619.77,5.58,NA,No censorship,NA,Legal,"Legal, surgery required"
